# Imports and Constants

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
import sklearn
import skelm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, RFE, SequentialFeatureSelector

from saving_outputs import *
from load_data import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *


SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
labels = {'vis' : np.array(classes*nb_runs), 'aud' : np.array(classes*nb_runs)}
labels_same = np.array(classes*nb_runs)

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                              (["vis"], ["PT_L", "PT_R"]),
                              (["aud"], ["V5_L", "V5_R"]),
                              (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                             (["vis", "aud"], ["PT_L", "PT_R"])]

# Analyses

In [ ]:
from_who = "our"   # 2 possibilities : "mohameds" or "our"
voxel_sizes = ["2"] # 2 possibilities : "2" or "3"
use_t_maps = True # use t-maps or beta-maps
correction = "_Tcorrected" # a t-value has been used to create ROIs or no
#radius_mask = "7"
radiuses = range(5,6)

#masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+radius_mask

std_scaler = sklearn.preprocessing.StandardScaler()

classifiers = {
    'svm':sklearn.svm.SVC(C=1, random_state=SEED),
    #'LR':sklearn.linear_model.LogisticRegression(random_state=SEED)
    #'ELM':skelm.ELMClassifier(random_state=SEED, n_neurons=100, alpha=1, ufunc='sigm')
    }
classifier = classifiers["svm"]

param_grids = {
    'svm':{
        #'svm__C': [0.1, 1],
        #'svm__gamma': [1, 0.1],
        'svm__kernel': ['linear']},
    'LR':{
        'LR__C': [1]},
    'ELM':{
        'ELM__n_neurons': [100]}
    }

cv_scheme = list()
for i in range(11):
    full_idx = range(44)
    idx_te = [i*4,i*4+1,i*4+2,i*4+3]
    idx_tr = [x for x in full_idx if x not in idx_te]
    tr_te_splits = [idx_tr, idx_te]
    cv_scheme.append(tr_te_splits)


In [ ]:
out_folders = list()
acc_group_combination = dict()
voxel_amounts = dict()
for voxel_size in voxel_sizes:
    acc_group_combination[voxel_size]= dict((radius,None) for radius in radiuses)
    voxel_amounts[voxel_size]= [None]*len(radiuses)

start_time = time.time()

maps_folder="brain_maps/"+from_who+"_maps"+"_"+voxel_size
masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+str(5)+correction
maps_masked, masks_exist = load_full_data(subjects_ids, len(classes), nb_runs, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"), use_t_maps=False)
voxel_amounts[voxel_size][0] = maps_masked[0]["vis"][0]["V5_R"].shape[1]

for k in ["a"]:
    for samples in [15]:

        def k_selector(n_voxel): return int(k*n_voxel)
        
        for i, radius in enumerate(radiuses) :
            for voxel_size in voxel_sizes :

                
                #print(voxel_amounts[voxel_size][i])

                models = dict()
                #selector = SelectKBest(k=k_selector(voxel_amounts[voxel_size][i]), score_func=f_classif)
                for name in classifiers:
                    pipeline = Pipeline([('scaler', std_scaler), 
                                        #('sfs', (SequentialFeatureSelector(skelm.ELMClassifier(random_state=SEED, n_neurons=100, alpha=1, ufunc='sigm'), n_features_to_select=k_selector(voxel_amounts[voxel_size][i])))), 
                                        #('selection', selector),
                                        (name, classifiers[name])])
                    GS = GridSearchCV(pipeline, param_grids[name], cv=cv_scheme, n_jobs=3)
                    models[name] = GS

                decoder = Decoder(n_perm=n_individual_perms, models=models, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
                decoder.set_masks_exist(masks_exist)

                # launch analyses
                confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
                confusion_matrixes_within = change_confusion_matrixes_org(confusion_matrixes_within[0], subjects_ids, models.keys())

                confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
                confusion_matrixes_cross = change_confusion_matrixes_org(confusion_matrixes_cross[0], subjects_ids, models.keys())

                # saving results
                type_maps = "_t_maps_" if use_t_maps else "_beta_maps_"
                out_directory = "out/"+from_who+"_"+type_maps+voxel_size+"_radius="+str(radius)+"_"+str(classifier)+"selecting_"+str(k)+"%/"
                create_directory(out_directory)
                out_folders.append(out_directory)

                save_dicts(out_directory + "confusion_matrixes_within.csv", confusion_matrixes_within["svm"],
                        list(confusion_matrixes_within["svm"][0].keys()), subjects_ids)
                acc_within = compute_metric(out_directory, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)

                save_dicts(out_directory + "confusion_matrixes_cross.csv", confusion_matrixes_cross["svm"],
                        list(confusion_matrixes_cross["svm"][0].keys()), subjects_ids)

                acc_cross = compute_metric(out_directory, subjects_ids, {'name' : 'cross', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)

                within_modality_group_results = average_dicos(acc_within)
                save_dicts(out_directory + "group_scores_within.csv", [within_modality_group_results],
                        list(within_modality_group_results.keys()), [0])
                acc_group_combination[voxel_size][radius] = within_modality_group_results

                cross_modality_group_results = average_dicos(acc_cross)
                save_dicts(out_directory + "group_scores_cross.csv", [cross_modality_group_results],
                        list(cross_modality_group_results.keys()), [0])
                acc_group_combination[voxel_size][radius].update(cross_modality_group_results)


                duration = time.time()-start_time
                print(" iteration with k="+str(k)+" done in "+str(duration)+" seconds")

# Plot scores depending on parameters

In [ ]:
colors = {"2":"tab:green","3":"tab:orange"}
plt_directory = "plots/"+from_who+"_"+type_maps+voxel_size+"_radius="+str(radius)+"_"+str(classifier)+"smote"+str(samples)+"kn"+str(kn)+"/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)
scores_var = dict()

for i, analysis in enumerate(acc_group_combination["2"][5]):
    # faire le plot
    for voxel_size in  voxel_sizes :
        scores = [acc_group_combination[voxel_size][radius][analysis] for radius in radiuses]
        scores_var[analysis] = np.var(scores*100)
        print(analysis)
        print((max(scores)-min(scores))*100)
        plt.plot(radiuses, scores, '-o', color = colors[voxel_size], label = voxel_size+" mm")
        plt.axhline(0.25, color="gray", alpha=0.5)
    title = plotter.generate_title("Accuracy",analysis,0)
    plt.ylim(0.2,0.5)
    plotter.save(title+ ", depending on ROI radius","","mean accuracy","radius [mm]",legend=True)

    for voxel_size in  voxel_sizes :
        scores = [acc_group_combination[voxel_size][radius][analysis] for radius in radiuses]
        #n_voxels = list(map(k_selector, np.array(voxel_amounts[voxel_size])))
        plt.plot(np.array(voxel_amounts[voxel_size]), scores, '-o', color = colors[voxel_size], label = voxel_size+" mm")
        #print("number of voxels : "+str(voxel_amounts[voxel_size]))
        #print("number of voxels selected : "+str(n_voxels))
        plt.axhline(0.25, color="gray", alpha=0.5)
    title = plotter.generate_title("Accuracy",analysis,0)
    plt.ylim(0.2,0.5)
    plotter.save(title+", depending on amount of voxels","","mean accuracy","voxel amount",legend=True)

print(scores_var)

In [ ]:
print([(an, scores_var[an]) for an in scores_var if scores_var[an] == max(scores_var.values())])
print([(an, scores_var[an]) for an in scores_var if scores_var[an] == min(scores_var.values())])

# Plot score and score variance for different k's
### (voxel_size and radius kept constant)

In [ ]:
#out_folders = list()
# subjects_ids = range(1, 24)
# for radius in radiuses:
#     for voxel_size in voxel_sizes:
#         out_folders.append("out/"+from_who+"_"+voxel_size+"_radius="+str(radius)+"_"+str(classifier)+"/")
#         print(out_folders)

# print(voxel_amounts[voxel_size][0])    
use_t_maps = True # use t-maps or beta-maps
out_folders = ["out/our_t_maps__avgtest_LogisticRegression(random_state=0)_3_radius=10/",
               "out/our_t_maps__avgtest_SVC(random_state=0)_3_radius=10/",
               "out/our_t_maps_LogisticRegression(random_state=0)_3_radius=10/",
               "out/our_t_maps_SVC(random_state=0)_3_radius=10/"]

# for folder in out_folders:

#     acc_within = compute_metric(folder, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "within", masks_exist, len(classes), ret = True)
#     acc_cross = compute_metric(folder, subjects_ids, {'name' : 'accuracy', 'function':accuracy}, "cross", masks_exist, len(classes), ret = True)

#     compute_accuracy_variance(folder, "within")
#     compute_accuracy_variance(folder, "cross")
    
labels = ["avg LR", "avg SVM", "LR", "SVM"]

plotter = Plotter("plots/comparing_avgtest_t_maps"+"_".join(labels), subjects_ids)
plotter.plot_tests_scores_from_different_folders(out_folders, labels, "average of 2 tests set", "technique")
plotter.plot_accuracy_var_from_different_folders(out_folders, labels)